In [1]:
#imports
import numpy as np
import pandas as pd
import random
import string

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import os
import csv
import datetime
from datetime import timedelta,datetime, date

from scipy import stats
from scipy.sparse import csr_matrix

import sklearn
from sklearn import metrics

import implicit
from implicit.nearest_neighbours import bm25_weight
from implicit.als import AlternatingLeastSquares

In [2]:
cu = pd.read_parquet("customers.parquet")
tr = pd.read_parquet("transactions_2_cleaned.parquet")

In [3]:
exptypes = tr.EXP_TYPE.unique()
prodtypes = cu.PROD_TYPE.unique()

cutoprod = {c:v.PROD_TYPE for (c,v) in cu.groupby("CUST_ID")}

In [4]:
grp = tr.loc[:, ["CUST_ID", "EXP_TYPE", "AMOUNT"]].groupby("CUST_ID")
l = []
lcu = []

for (k,v) in grp:
  df2 = v.groupby("EXP_TYPE").apply(lambda x: pd.Series([x.shape[0], 
    x.AMOUNT.sum()], index= ["n", "AMOUNT"]))
  df2.loc[:, "AMOUNT_prop"] = df2.AMOUNT / df2.AMOUNT.sum()
  df2.loc[:, "n_prop"] = df2.n / df2.n.sum()
  df2.loc[:, "AMOUNT_per_n"] = [0.0 if np.isnan(x) else x  
                                for x in df2.AMOUNT / df2.n]
  l.append(df2)
  lcu.append(k)

In [5]:
nc = len(exptypes)

datax = np.zeros((len(l), 3*len(exptypes)))
for (i,v) in enumerate(l):
  datax[i, range(nc)] = v.AMOUNT_prop
  datax[i, range(nc, nc + nc)] = v.n_prop
  datax[i, range(2*nc, 2*nc + nc)] = v.AMOUNT_per_n
  
datax_ap = datax[:, range(nc)]
datax_np = datax[:, range(nc, 2*nc)]
datax_apn = datax[:, range(2*nc, 3*nc)]

datay = np.array([ np.array([float((p == cutoprod[cu]).any())  for cu in lcu]) 
         for p in prodtypes]).transpose()

https://github.com/benfred/implicit/issues/125 

In [141]:
def prepare_model(datax):
  model = AlternatingLeastSquares(factors=34, regularization=0.05)
  datax2 = csr_matrix(datax)
  csrdatax = bm25_weight(datax2.T.tocsr(), K1=100, B=0.8).T.tocsr()
  model.fit(csrdatax, False)
  return(model)

def findsimilar_users(model, datax, newdata):
  newi = datax.shape[0]
  datax2 = csr_matrix(np.vstack([datax, newdata]))
  csrdatax = bm25_weight(datax2.T.tocsr(), K1=100, B=0.8).T.tocsr()
  model.partial_fit_users([newi], csr_matrix(datax2[[newi], :]))
  return(model.similar_users(newi, 10, [newi]))

#def average_products(datay, su):

In [142]:
model = prepare_model(datax)

In [143]:
su = findsimilar_users(model, datax, datax[12, :])
print(su)

(array([5033, 4372, 4427, 5109, 5314, 5913, 4984, 5932, 5116, 4723],
      dtype=int32), array([0.96125174, 0.9578122 , 0.9543303 , 0.95311785, 0.95015043,
       0.9498622 , 0.9481703 , 0.94607836, 0.93609583, 0.9333036 ],
      dtype=float32))


In [144]:
datay[su[0],:]


array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [8]:
def findsimilar_users2(datax, newdata):
  model = AlternatingLeastSquares(factors=34, regularization=0.05)
  newi = datax.shape[0]
  datax2 = csr_matrix(np.vstack([datax, newdata]))
  csrdatax = datax2#bm25_weight(datax2.T.tocsr(), K1=2, B=0.8).T.tocsr()
  model.fit(csrdatax, False)
  return(model.similar_users(newi, 10, [newi]))  

In [9]:
su = findsimilar_users2(datax, datax[[12], :])
print(su)

/home/icarus/.local/lib/python3.9/site-packages/implicit/utils.py:26: UserWarning: OpenBLAS detected. Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


(array([  12,  355, 5610, 1151, 5164,  696, 6847, 5913, 3686, 1058],
      dtype=int32), array([1.        , 0.9998425 , 0.9988973 , 0.9984989 , 0.9981169 ,
       0.99806595, 0.9975769 , 0.9975633 , 0.99737585, 0.99725366],
      dtype=float32))


In [10]:
datay[su[0],:]


array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 1., 0., 0., 0., 0., 0., 0., 0.]])

In [11]:
def findsimilar_users3(datax, newdata):
  model = AlternatingLeastSquares(factors=34, regularization=0.05)
  newi = datax.shape[0]
  datax2 = csr_matrix(np.vstack([datax, newdata]))
  csrdatax = bm25_weight(datax2, K1=2, B=0.8).tocsr()
  model.fit(csrdatax, False)
  return(model.similar_users(newi, 10, [newi]))  

In [12]:
su = findsimilar_users3(datax, datax[[12], :])
print(su)

(array([  12,  355, 3105, 3145, 3863, 3738, 5610, 2606, 1999, 2364],
      dtype=int32), array([1.        , 0.9999347 , 0.99988836, 0.9858127 , 0.9845693 ,
       0.9838061 , 0.98125863, 0.9778415 , 0.9736157 , 0.97096896],
      dtype=float32))


In [13]:
datay[su[0],:]


array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.]])

In [14]:
su = findsimilar_users3(datax_ap, datax_ap[[12], :])
print(su)

(array([  12, 3105,  355, 5039,  334, 2364, 3686, 5913, 3145, 1999],
      dtype=int32), array([1.        , 0.99833894, 0.9972507 , 0.974794  , 0.972698  ,
       0.97139215, 0.97120965, 0.9711225 , 0.9676098 , 0.96215695],
      dtype=float32))


In [15]:
datay[su[0],:]


array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])